In [53]:
from trueskill import Rating, rate
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import json

In [54]:
def print_commanders(commanders, skill_estimate_k = 3):
    commanders = {k: {"rating": v, "skill estimate": skill_estimate(v, k = skill_estimate_k)} for k, v in commanders.items()}
    ratings = [v["skill estimate"] for v in commanders.values()]
    rankings = sorted(range(len(ratings)), key=lambda i: ratings[i], reverse=True)
    for i, rank in enumerate(rankings):
        print(f"{i + 1}. {list(commanders.keys())[rank]} - {list(commanders.values())[rank]}")

In [55]:
def skill_estimate(rating, k = 3):
    """
    Calculate the conservative skill estimate for a given rating.
    This is the mu value minus k times the sigma value.
    """
    return float(rating.mu) - k * float(rating.sigma)

In [56]:
with open("results.json") as results_file:
    results = json.load(results_file)

In [57]:
commanders = set()
players = set()

for match in results:
    commanders.update([winner["commander"] for winner in match["winners"]])
    commanders.update([loser["commander"] for loser in match["losers"]])
    players.update([winner["player"] for winner in match["winners"]])
    players.update([loser["player"] for loser in match["losers"]])

print(commanders)
print(players)


commanders = {commander: Rating() for commander in commanders}
players = {player: Rating() for player in players}

{"Atraxa, Praetors' Voice", 'Neera, Wild Mage', 'Astarion, the Decadent', 'Gandalf, Westward Voyager', 'Aminatou, Veil Piercer', 'Kianne, Corrupted Memory', 'Bria, Riptide Rogue', 'Ulalek, Fused Atrocity', 'Dionus, Elvish Archdruid', "Y'shtola, Night's Blessed", 'Hazel of the Rootbloom', 'Vnwxt, Verbose Host', 'Zedruu the Greathearted', 'Valgavoth, Harrower of Souls', 'Clavileño, First of the Blessed', 'Satya, Aetherflux Genius', 'Davros, Dalek Creator', 'Edgar Markov', 'Umbris, Fear Manifest', 'Omo, Queen of Vesuva', 'Tom Bobadil', 'Saruman, the White Hand', 'Maha, Its Feathers Night', 'Lord Windgrace', 'Rakdos, Lord of Riots', "Temmet, Naktamun's Will", 'Lathril, Blade of the Elves', 'Cloud, Ex-SOLDIER', "Eshki, Temur's Roar", 'Rona, Herald of Invasion // Rona, Tolarian Obliterator', 'Frodo, Adventurous Hobbit // Sam, Loyal Attendant', 'Nekusar, the Mindrazer', "Otharri, Suns' Glory", 'Krenko, Mob Boss', 'The Jolly Balloon Man', 'Ojer Axonil, Deepest Might'}
{'Bettina', 'Julian', 'Se

In [58]:
# commanders = players

In [59]:
print_commanders(commanders)

1. Atraxa, Praetors' Voice - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
2. Neera, Wild Mage - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
3. Astarion, the Decadent - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
4. Gandalf, Westward Voyager - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
5. Aminatou, Veil Piercer - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
6. Kianne, Corrupted Memory - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
7. Bria, Riptide Rogue - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
8. Ulalek, Fused Atrocity - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
9. Dionus, Elvish Archdruid - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
10. Y'shtola, Night's Blessed - {'rating': trueskill.Rating(mu=25

In [60]:
# commanders = players

In [61]:
commanders_vis = pd.DataFrame(columns=["Match", "Commander", "TrueSkill Rating", "TrueSkill StDev"])

for i, match in enumerate(results):
    # print("\nMatch ", i + 1, " on ", match["date"], ":", sep="")
    commander_winners = [winner["commander"] for winner in match["winners"]]
    commander_losers = [loser["commander"] for loser in match["losers"]]
    # commander_winners = [winner["player"] for winner in match["winners"]]
    # commander_losers = [loser["player"] for loser in match["losers"]]
    commanders_in_match = commander_winners + commander_losers
    
    # print("Ratings before match:")
    # print_commanders({commander: commanders[commander] for commander in commanders_in_match})

    # print("\nWinners:", end=" ")
    # for winner in commander_winners[:(len(commander_winners)-1)]:
    #     print(winner, end="; ")
    # print(commander_winners[-1])
    # print("Losers:", end=" ")
    # for loser in commander_losers[:(len(commander_losers)-1)]:
    #     print(loser, end="; ")
    # print(commander_losers[-1])
    
    teams = [{commander: commanders[commander]} for commander in commander_winners] + \
             [{commander: commanders[commander]} for commander in commander_losers]
    ranks = [0] * len(commander_winners) + [1] * len(commander_losers)
    result = rate(teams, ranks=ranks)

    for team in result:
        for commander in team:
            commanders[commander] = team[commander]

    # print("\nRatings after match:")
    # print_commanders({commander: commanders[commander] for commander in commanders_in_match})

    current_match = pd.DataFrame({
        "Match": i + 1,
        "Commander": commanders_in_match,
        "TrueSkill Rating": [commanders[commander].mu for commander in commanders_in_match],
        "TrueSkill StDev": [commanders[commander].sigma for commander in commanders_in_match]
    })

    if commanders_vis.empty:
        commanders_vis = current_match
    else:
        previous_matches = commanders_vis[(commanders_vis["Match"] == i) & (~commanders_vis["Commander"].isin(commanders_in_match))]
        previous_matches.loc[:,"Match"] = i + 1

        commanders_vis = pd.concat([commanders_vis, current_match, previous_matches], ignore_index=True)

In [62]:
print_commanders(commanders)

1. Lord Windgrace - {'rating': trueskill.Rating(mu=32.064, sigma=5.942), 'skill estimate': 14.236644655978836}
2. Vnwxt, Verbose Host - {'rating': trueskill.Rating(mu=23.802, sigma=3.206), 'skill estimate': 14.18514346813419}
3. Valgavoth, Harrower of Souls - {'rating': trueskill.Rating(mu=26.925, sigma=4.522), 'skill estimate': 13.358179375747474}
4. Saruman, the White Hand - {'rating': trueskill.Rating(mu=31.614, sigma=6.431), 'skill estimate': 12.320638257397857}
5. Ojer Axonil, Deepest Might - {'rating': trueskill.Rating(mu=26.656, sigma=4.904), 'skill estimate': 11.94444153356054}
6. Y'shtola, Night's Blessed - {'rating': trueskill.Rating(mu=30.547, sigma=6.312), 'skill estimate': 11.610664287788854}
7. Bria, Riptide Rogue - {'rating': trueskill.Rating(mu=25.374, sigma=4.632), 'skill estimate': 11.4771499201016}
8. Kianne, Corrupted Memory - {'rating': trueskill.Rating(mu=26.730, sigma=5.120), 'skill estimate': 11.369271470120268}
9. Nekusar, the Mindrazer - {'rating': trueskill.R

In [63]:
def generate_color_palette(n, alpha=1.0):
    return [
        f"hsla({h}, 70%, 50%, {alpha})"
        for h in range(0, 360, int(360 / n))
    ]

In [64]:
default_selection = ["Lord Windgrace", "Atraxa, Praetors' Voice", "Nekusar, the Mindrazer"]

In [65]:
commanders_vis['Match'] = commanders_vis['Match'].astype(int)
commanders_vis = commanders_vis.sort_values(['Commander', 'Match'])

fig = go.Figure()

commanders = commanders_vis['Commander'].unique()
base_colors = generate_color_palette(len(commanders), alpha=1.0)
band_colors = generate_color_palette(len(commanders), alpha=0.1)

# Get distinct colors for each commander using Plotly's qualitative color palette
color_map_line = dict(zip(commanders, base_colors))
color_map_band = dict(zip(commanders, band_colors))

for commander in commanders:
    data = commanders_vis[commanders_vis['Commander'] == commander].copy()
    data = data.sort_values('Match')

    color_line = color_map_line[commander]
    color_band = color_map_band[commander]
    # color_rgba = color.replace('rgb', 'rgba').replace(')', ', 0.15)')  # Transparent fill

    upper = data['TrueSkill Rating'] + data['TrueSkill StDev']
    lower = data['TrueSkill Rating'] - data['TrueSkill StDev']

    # Line trace
    fig.add_trace(go.Scatter(
        x=data['Match'], y=data['TrueSkill Rating'],
        mode='lines',
        name=commander,
        legendgroup=commander,
        line=dict(width=2, color=color_line),
        hoverinfo='x+y+name',
        visible='legendonly' if not commander in default_selection else True
    ))

    # Confidence band
    fig.add_trace(go.Scatter(
        x=pd.concat([data['Match'], data['Match'][::-1]]),
        y=pd.concat([upper, lower[::-1]]),
        fill='toself',
        fillcolor=color_band,
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo='skip',
        showlegend=False,
        legendgroup=commander,
        visible='legendonly' if not commander in default_selection else True
    ))

fig.update_layout(
    title='Commander TrueSkill Ratings with Confidence Bands',
    xaxis_title='Match',
    yaxis_title='TrueSkill Rating',
    template='plotly_white',
    legend_title='Commander',
    hovermode='x unified'
)

fig.show()

In [ ]:
k = 3

commanders_vis["estimated skill"] = commanders_vis["TrueSkill Rating"] - 3 * commanders_vis["TrueSkill StDev"]

fig = go.Figure()

base_colors = generate_color_palette(len(commanders), alpha=1.0)

# Get distinct colors for each commander using Plotly's qualitative color palette
color_map_line = dict(zip(commanders, base_colors))

for commander in commanders:
    data = commanders_vis[commanders_vis['Commander'] == commander].copy()
    data = data.sort_values('Match')

    color_line = color_map_line[commander]

    # Line trace
    fig.add_trace(go.Scatter(
        x=data['Match'], y=data['estimated skill'],
        mode='lines',
        name=commander,
        legendgroup=commander,
        line=dict(width=2, color=color_line),
        hoverinfo='x+y+name',
        visible='legendonly' if not commander in default_selection else True
    ))

fig.update_layout(
    title='Commander estimated Ratings ',
    xaxis_title='Match',
    yaxis_title='TrueSkill Rating',
    template='plotly_white',
    legend_title='Commander',
    hovermode='x unified'
)

fig.show()